# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit

# Define

In [46]:
def IsNone(target):
    return (type(target) == type(None))

def ArraySign(input_array):
    # return +1, 0, -1 respect to positive, zero, negtive
    return 1.*(input_array>0) - 1.*(input_array<0)

def CutValue(input_array, cut_value):
    output = np.abs(input_array)
    output = ArraySign(input_array) * (output * (output < cut_value) + cut_value * (output >= cut_value))
    return output

def SquareSum(input_array):
    return np.square(input_array).sum()

def OverPenalty(input_value, rate = 0.1, threshold=0.):
    output = np.abs(input_value) - threshold
    output *= (output > 0)
    output *= rate * ArraySign(input_value)
    return output

def RowOperate(matrix, threshold = 0.1**15):
    reduced_matrix = np.array(matrix)
    filtered_matrix = np.array(matrix)
    shape = matrix.shape # matrix size
    mask = np.ones(shape)
    pivots = -1*np.ones((min(shape)), dtype = np.int) # store pivots, # of pivots <= min(rows, columns)
    for t in range(len(pivots)):
        filtered_matrix = reduced_matrix * mask # filter
        if np.abs(filtered_matrix).max() < threshold:
            break
        
        pivot_row, pivot_col = np.unravel_index(np.abs(filtered_matrix).argmax(), shape) # pivot row, pivot column
        reduced_matrix[pivot_row] /= reduced_matrix[pivot_row][pivot_col]
        multi = np.array(reduced_matrix[:, pivot_col])
        multi[pivot_row] = 0.
        reduced_matrix -= np.dot(multi.reshape((shape[0], 1)), reduced_matrix[pivot_row].reshape((1, shape[1])))
        mask[pivot_row] = 0.
        mask[:, pivot_col] = 0.
        pivots[pivot_row] = pivot_col # the column-index of pivot_row-th row is pivot_col
    
    reduced_matrix = reduced_matrix[pivots != -1,:]
    pivots = pivots[pivots != -1]
    
    return reduced_matrix, pivots

def McmcNormal(points, drop_times = 10, mean=0., std=1.):
    output = np.random.normal(mean, std/np.sqrt(points[0].size), points.shape[1:])
    if drop_times>1:
        for t in range(1, drop_times):
            candicate = np.random.normal(mean, std/np.sqrt(points[0].size), points.shape[1:])
            candicate_distance = np.sqrt(np.square(np.subtract(points, candicate)).sum(axis=tuple(np.arange(1, len(points.shape))))).min()
            # distance of candicate to target
            output_distance = np.sqrt(np.square(np.subtract(points, output)).sum(axis=tuple(np.arange(1, len(points.shape))))).min()
            # distance of currently output to target
            if np.random.rand()*output_distance < candicate_distance:
                output = np.array(candicate)
    
    return output

# Data

class Data():
    def __init__(self, inputs=np.zeros((0,0)), labels=np.zeros((0,0)), weight = None):
        self.SetData(inputs, labels, weight)
    
    def SetData(self, inputs=np.zeros((0,0)), labels=np.zeros((0,0)), weight = None):
        if len(inputs) != len(labels):
            raise ValueError("num_datums error, #inputs != #labels.")
        
        self.inputs = inputs
        self.labels = labels
        if IsNone(weight):
            self.weight = np.ones((inputs.shape[0]))
        elif weight.shape != (len(inputs)):
            self.weight = np.ones((inputs.shape[0]))
            print("WARNING : weight shape error, set uniform weight.")
        elif weight.sum() <= 0:
            self.weight = np.ones((inputs.shape[0]))
            print("WARNING : get non-positive weight sum, set uniform weight.")
        else:
            self.weight = weight
        
        self.weight /= self.weight.sum()
    
    def GetNumDatums(self):
        return len(self.inputs)
    
    def GetDatumSize(self):
        # return size of input and label
        return (self.inputs.shape[1], self.labels.shape[1])

# Data end

# VariableArray

class VariableArray():
    def __init__(self, size, cwise_step_initial=0.1):
        self.value = np.random.normal(0., 1., size) # array value
        self.total_deri = np.zeros(self.value.shape) # total derivative, used to descent
        self.last_total_deri = None # last total derivative
        self.moving = np.zeros(self.value.shape) # moving array
        self.cwise_step = cwise_step_initial*np.ones(self.value.shape) # component-wise step
        
        self.regulariz_rate = 0.
        self.regulariz_margin = 0.
    
    def SetValue(self, input_value, cwise_step_initial=0.1):
        self.value = np.array(input_value) # array value
        self.total_deri = np.zeros(self.value.shape) # total derivative, used to descent
        self.last_total_deri = None # last total derivative
        self.moving = np.zeros(self.value.shape) # moving array
        self.cwise_step = cwise_step_initial*np.ones(self.value.shape) # component-wise step
    
    def SetDeri(self, input_value):
        if input_value.shape != self.total_deri.shape:
            raise ValueError("input_value shape error")
        
        self.last_total_deri = np.array(self.input_value)
        self.total_deri = np.array(input_value)
    
    def DeriModify(self, input_value):
        if input_value.shape != self.total_deri.shape:
            raise ValueError("input_value shape error")
        
        self.total_deri += input_value
    
    def SetRegularizer(self, rate, margin):
        self.regulariz_rate = max(rate, 0.)
        self.regulariz_margin = max(margin, 0.)
    
    def AddRow(self, input_row, cwise_step_initial=0.1):
        self.value = np.append(self.value, np.array([input_row]), axis = 0)
        self.total_deri = np.append(self.total_deri, np.zeros((1,) + input_row.shape), axis = 0)
        self.last_total_deri = None
        self.moving = np.zeros(self.value.shape)
        self.cwise_step = np.append(self.cwise_step, cwise_step_initial * np.ones((1,) + input_row.shape), axis = 0)
    
    def AddColumn(self, input_column, cwise_step_initial=0.1):
        self.value = np.append(self.value, np.array([input_column]).T, axis = 1)
        self.total_deri = np.append(self.total_deri, np.zeros(input_column.shape + (1,)), axis = 1)
        self.last_total_deri = None
        self.moving = np.zeros(self.value.shape)
        self.cwise_step = np.append(self.cwise_step, cwise_step_initial * np.ones(input_column.shape + (1,)), axis = 1)
    
    def ResetCwiseStep(self, input_cwise_step):
        self.cwise_step = input_cwise_step * np.ones(self.cwise_step.shape)
    
    def Regularize(self):
        if self.regulariz_rate != 0:
            self.total_deri -= OverPenalty(self.value, self.regulariz_rate, self.regulariz_margin)
    
    def Descent(self, step=1., method="normal", move_max=1.):
        self.Regularize()
        if method == "normal":
            self.moving = self.total_deri * step
        elif method == "Rprop":
            self.moving = ArraySign(self.total_deri)
            self.movint_return = ArraySign(self.total_deri*self.last_total_deri)
            self.cwise_step *= 1.2*(self.movint_return>0) + 1.*(self.movint_return==0) + 0.5*(self.movint_return<0)
            self.moving *= self.cwise_step
        else:
            raise ValueError("descent method error")
        
        self.moving = -1*CutValue(self.moving, move_max)
        self.value += self.moving

# VariableArray end

# Activation functions defined start

class Identity():
    def Forward(self, flow_in):
        return flow_in
    
    def Diff(self, flow_in):
        return np.ones(flow_in.shape, dtype = np.float64)
    
    def Backward(self, flow_in, back_flow):
        return self.Diff(flow_in) * back_flow

class Sigmoid():
    def Forward(self, flow_in):
        return expit(flow_in)
    
    def Diff(self, flow_in):
        return expit(flow_in)*expit(-flow_in)
    
    def Backward(self, flow_in, back_flow):
        return self.Diff(flow_in) * back_flow

class Hypertan():
    def Forward(self, flow_in):
        return np.tanh(flow_in)
    
    def Diff(self, flow_in):
        flow_in[(flow_in<-100)&(flow_in>100)] = 100 # cut value out of [-100, 100] to 100, cosh(-100) = cosh(100)
        return 1. / np.square(np.cosh(flow_in))
    
    def Backward(self, flow_in, back_flow):
        return self.Diff(flow_in) * back_flow

class SoftSign():
    def Forward(self, flow_in):
        return ArraySign(flow_in)*(1. - 1./(np.abs(flow_in) + 1.))
    
    def Diff(self, flow_in):
        return 1. / np.square(np.abs(flow_in) + 1.)
    
    def Backward(self, flow_in, back_flow):
        return self.Diff(flow_in) * back_flow

class Relu():
    def Forward(self, flow_in):
        return flow_in*(flow_in>0)
    
    def Diff(self, flow_in):
        return 1.*(flow_in>0)
    
    def Backward(self, flow_in, back_flow):
        return self.Diff(flow_in) * back_flow

class LeakyRelu():
    def __init__(self, alpha = 0.1):
        self.alpha = alpha
    
    def Forward(self, flow_in):
        return flow_in*(flow_in>0) + self.alpha*flow_in*(flow_in<0)
    
    def Diff(self, flow_in):
        return 1.*(flow_in>0) + self.alpha*(flow_in<0)
    
    def Backward(self, flow_in, back_flow):
        return self.Diff(flow_in) * back_flow

class SoftPlus():
    def Forward(self, flow_in):
        return np.log(1. + np.exp(flow_in))
    
    def Diff(self, flow_in):
        return expit(flow_in)
    
    def Backward(self, flow_in, back_flow):
        return self.Diff(flow_in) * back_flow

class Selu():
    def __init__(self):
        self.ahpha = 1.05071
        self.beta = 1.67326
    
    def Forward(self, flow_in):
        return self.ahpha*(flow_in*(flow_in>=0) + self.beta*(np.exp(flow_in) - 1)*(flow_in<0))
    
    def Diff(self, flow_in):
        return self.ahpha*(1.*(flow_in>=0) + self.beta*np.exp(flow_in)*(flow_in<0))
    
    def Backward(self, flow_in, back_flow):
        return self.Diff(flow_in) * back_flow

class Softmax():
    def Forward(self, flow_in):
        output = flow_in - flow_in.max(axis=1).reshape(-1,1)
        output = np.exp(output)
        output /= output.sum(axis=1).reshape(-1,1)
        return output
    
    def Backward(self, flow_in, back_flow):
        flow_out = self.Forward(flow_in) # result of self.trans
        return flow_out*back_flow - flow_out*((flow_out*back_flow).sum(axis=1).reshape(-1,1))

# Activation functions defined end

# Layer

class Layer():
    def __init__(self, num_unit, activationFunction):
        if type(activationFunction) == type:
            raise TypeError("activationFunction should be a class. eg: Use 'Sigmoid()', not 'Sigmoid'")
        
        self.num_unit = num_unit
        self.activationFunction = activationFunction
        self.linear = VariableArray((0, self.num_unit)) # linear weights working before active function
        self.bias = VariableArray((1, self.num_unit)) # bias working before active function
        self.flow_in = np.zeros((0, self.num_unit))
        self.flow_out = np.zeros((0, self.num_unit))
    
    def Forward(self, flow_in):
        self.flow_in = np.dot(self.linear.value, flow_in) + self.bias.value
        self.flow_out = self.activationFunction.Forward(self.flow_in)
    
    def Backward(self, back_flow, layer_source):
        deri = self.activationFunction.Backward(self.flow_in, back_flow)
        self.linear.SetDeri(np.dot(layer_source.T, deri))
        self.bias.SetDeri(np.sum(deri, axis=0).reshape(1, -1))
        deri = np.dot(deri, self.linear.value.T)
        return deri
    
    def SetRegularizer(self, rate, margin):
        self.linear.SetRegularizer(rate, margin)
        self.bias.SetRegularizer(rate, margin)
    
    def Descent(self, step, method):
        self.linear.Descent(step, method)
        self.bias.Descent(step, method)
    
    def ResetCwiseStep(self, new_cwise_step):
        self.linear.ResetCwiseStep(new_cwise_step)
        self.bias.ResetCwiseStep(new_cwise_step)
    
    def GetDimension(self):
        return self.linear.value.size + self.bias.value.size

# Layer end

# Loss function

class LossFunction():
    def __init__(self, method):
        self.method = method
    
    def SetMethod(self, method):
        self.method = method
    
    def GetLoss(self, inference, target):
        if self.method == "r2":
            output = SquareSum(inference - target)/SquareSum(target-target.sum(axis=0))
        elif self.method == "cross entropy":
            output = -target*np.log(inference).sum(axis=1).mean()
        else:
            raise ValueError("loss function method should be 'r2', 'cross entropy'")
        
        return output
    
    def Backward(self, inference, target):
        if self.method == "r2":
            output = 0.5*(inference - target)/SquareSum(target-target.sum(axis=0))
        elif self.method == "cross entropy":
            output = -(target/inference)/target.shape[0]
        else:
            raise ValueError("loss function method should be 'r2', 'cross entropy'")
        
        return

class DogikoNeuralNetwork():
    def __init__(self):
        self.hiddenLayerList = []
        self.outputFunction = None
        self.outputLayer = None
        self.trainData = Data()
        self.validData = Data()
        self.testData = Data()
        self.regulariz_rate = 0.
        self.regulariz_margin = 0.
        self.has_build = False
    
    def SetLossFunction(self, method):
        if method not in ["r2", "cross entropy"]:
            raise ValueError("loss function method should be 'r2', 'cross entropy'")
        
        self.lossFunction = LossFunction(method)
    
    def SetRegularizer(self, rate, margin):
        if rate < 0.:
            print("WARNING : regulariz_rate error, get negative value, setting to 0.")
        
        if margin < 0.:
            print("WARNING : regulariz_margin error, get negative value, setting to 0.")
        
        self.regulariz_rate = max(rate, 0.)
        self.regulariz_margin = max(margin, 0.)
        
        if self.has_build:
            pass
    
    def SetTrainData(self, inputs, labels, weight = None):
        self.trainData.SetData(inputs, labels, weight)
    
    def SetValidData(self, inputs, labels, weight = None):
        self.validData.SetData(inputs, labels, weight)
    
    def SetTestData(self, inputs, labels, weight = None):
        self.testData.SetData(inputs, labels, weight)
    
    def AddHiddenLayer(self, num_unit, activationFunction):
        self.hiddenLayerList.append(Layer(num_unit, activationFunction))
        if self.has_build:
            print("WARNING : a hidden layer added after build, please re-build model or set related value manually.")
    
    def SetOutputFunction(self, activationFunction):
        # units of output layer is fixed as same as label size
        self.outputFunction = activationFunction
        if self.has_build:
            self.outputLayer.activationFunction = self.outputFunction

# --------------------------------------------





# --------------------------------------------
"""    
    def build(self):
        self.ln = len(self.ly) # amount of layers
        self.ly[0].w.assign_values(np.random.normal(0., 1., (self.ly[0].nn, self.xs)), self.csi)
        self.ly[0].b.assign_values(np.random.normal(0., 1., (self.ly[0].nn, 1)), self.csi)
        for l in range(1,self.ln):
            self.ly[l].w.assign_values(np.random.normal(0., 1., (self.ly[l].nn, self.ly[l-1].nn)), self.csi)
            self.ly[l].b.assign_values(np.random.normal(0., 1., (self.ly[l].nn, 1)), self.csi)
        
        if self.ly[-1].nn != self.ys: # cheak output size
            raise ValueError("output layer must has the same size with datum lables(answer)")
    
    def prediction(self, data_input):
        self.px = np.array(data_input) # prediction data input of last time predic
        if self.px.shape[1] != self.xs:
            raise ValueError("datum size error")
        
        self.ly[0].Forward(self.px.T)
        for l in range(1,self.ln):
            self.ly[l].Forward(self.ly[l-1].y)
        
        self.py = np.array(self.ly[-1].y.T) # prediction result of last time predict
        
        return self.py
    
    def descent(self, step = 1., descent_method = "normal"):
        for l in range(self.ln):
            self.ly[l].descent(step, descent_method, self.rg)
        
        if descent_method in ["Rprop", "Dogiko Rprop"]:
            self.max_cs = 0.
            for l in range(self.ln):
                self.max_cs = max(self.max_cs, self.ly[l].w.max_cs(), self.ly[l].b.max_cs())
    
    def evaluate(self, _input, labels):
        self.prediction(_input)
        if self.lf == "r2":
            return np.square(self.py - labels).mean()/labels.var(axis=0).mean()
        elif self.lf == "ce":
            return (-1*labels*np.log(self.py+0.0001)).sum(axis=1).mean()
        else:
            raise ValueError("loss function should be 'r2' or 'ce'")
    
    def gradient_get(self, _input, labels):
        self.prediction(_input)
        if self.lf == "r2":
            temp_derivative = 2*(self.py - labels).T/(labels.shape[0]*labels.var(axis=0).sum())
        elif self.lf == "ce":
            temp_derivative = -1*(labels/(self.py + 0.0001)).T/labels.shape[0]
        else:
            raise ValueError("loss function should be 'r2' or 'ce'")
        
        for l in range(self.ln-1, 0, -1):
            temp_derivative = self.ly[l].Backward(temp_derivative, self.ly[l-1].y)
        
        self.ly[0].Backward(temp_derivative, _input.T)
    
    def batch_fit(self, batch_input, batch_labels, step = 1., descent_method = "normal"):
        self.gradient_get(batch_input, batch_labels)
        self.descent(step, descent_method)
    
    def epoch_fit(self, batch_size = None, step = 1., descent_method = "normal"):
        if type(batch_size) == type(None):
            self.batch_fit(self.tx, self.ty, step, descent_method)
        elif type(batch_size) == int:
            if batch_size > 0:
                for b in range(np.ceil(self.tx.shape[0]/ batch_size).astype(np.int)):
                    self.batch_fit(self.tx[b*batch_size: (b+1)*batch_size],
                                   self.ty[b*batch_size: (b+1)*batch_size],
                                   step,
                                   descent_method
                                  )
            else:
                raise ValueError("batch_size should be positive int")
            
        else:
            raise ValueError("batch_size should be positive int")
    
    def train(self, times, batch_size = None, step = 1., descent_method = "normal", termination = [0,0,0.]):
        is_termination = False
        try:
            termination[2] > 12345 # test whether threshold is an real number (no mater int, float,.etc)
            termination[0] = int(termination[0])
            termination[1] = int(termination[1])
            if (termination[0] < termination[1]) and (termination[0] > 0):
                is_termination = True
                error_record = []
        except:
            pass
        
        for t in range(times):
            self.epoch_fit(batch_size, step, descent_method)
            if is_termination:
                if self.lf == "ce":
                    error_record.append(10*np.log10(1.000000001 - self.validation_accuracy()))
                else:
                    error_record.append(10*np.log10(self.validation_error()+0.000000001))
                # 0.000000001, bias for prevent error when log(0)
                
                if t >= (termination[1] - 1):
                    short_mean = sum(error_record[(-termination[0]):])/termination[0]
                    long_mean = sum(error_record[(-termination[1]):])/termination[1]
                    if (long_mean - short_mean) < termination[2]:
                        return t+1
        
        return times
            
    def accuracy(self, inference, target):
        if inference.shape != target.shape:
            raise ValueError("shape of inference and target non-equal")
            
        return (inference.argmax(axis=1) == target.argmax(axis=1)).sum()/inference.shape[0]
    
    def training_error(self):
        return self.evaluate(self.tx, self.ty)
    
    def training_accuracy(self):
        return self.accuracy(self.prediction(self.tx), self.ty)
    
    def validation_error(self):
        return self.evaluate(self.vx, self.vy)
    
    def validation_accuracy(self):
        return self.accuracy(self.prediction(self.vx), self.vy)
    
    def neuron_refined(self, l, reference_data = None, threshold = 0.01):
        # l : the # of layer
        # threshold : threshold for information contained of dimension be remaind
        if type(l) != int:
            raise TypeError("l should be the layer no. of hidden layer, an int between 0 to (neural_number - 2)")
        elif (l >= self.ln - 1) or (l < 0):
            raise ValueError("l should be the layer no. of hidden layer, an int between 0 to (neural_number - 2)")
        
        try:
            if ((threshold< 1) and (threshold>0)) or (type(threshold) == int):
                if (threshold > self.ly[l].nn-1):
                    raise ValueError("int threshold error : removed #neuron should less than currently #neuron")
                elif (threshold < -self.ly[l].nn) or (threshold==0):
                    return None
                    # do nothing if remove no #neuron (threshold=0) or want to remain #neuron more than currently
            else:
                raise ValueError("threshold : a value in (0, 1), or an nonzero int")
        except:
            raise ValueError("threshold : a value in (0, 1), or an nonzero int")
        
        if type(reference_data) == type(None):
            self.prediction(self.tx)
        else:
            self.prediction(reference_data)
        
        ym = self.ly[l].y.mean(axis=1).reshape((self.ly[l].nn,1)) # y (output of Layer) mean of each neurons
        yn = self.ly[l].y - ym # centralized y
        ab = np.dot(self.ly[l+1].w.v, ym) # Adjusted bias
        ev, em = np.linalg.eigh(np.dot(yn, yn.T)) # eigenvalues and eigenmatrix(with eigenvectors as columns)
        ir = ev/ev.sum() # info ratio for each eigenvector
        # op, pv :column operator result and pivots
        if (threshold< 1) and (threshold>0):
            op, pv = column_operate(em[:,ir > threshold])
        else:
            op, pv = column_operate(em[:,ir >= ir[ir.argsort()[threshold]]])
            
        nw = np.dot(self.ly[l+1].w.v, op) # new weight
        self.ly[l+1].b.assign_values(self.ly[l+1].b.v + (np.dot(self.ly[l+1].w.v, ym) -np.dot(nw, ym[pv])))
        self.ly[l+1].w.assign_values(nw) # l+1 weight should be rewrite after l+1 bias have been rewrite
        self.ly[l].w.assign_values(self.ly[l].w.v[pv])
        self.ly[l].b.assign_values(self.ly[l].b.v[pv])
        self.ly[l].nn = len(pv)
    
    def neuron_proliferate(self, proliferating_layer, proliferating_n = 1, output_weight_bound = 1.):
        if proliferating_layer not in range(self.ln):
            raise ValueError("proliferating_layer should be an int from 0 to (#layer-1)")
            
        if type(proliferating_n) != int:
            raise ValueError("proliferating_n should be int")
        
        if proliferating_n <= 0:
            raise ValueError("proliferating_n should be postive")
            
        if output_weight_bound < 0.:
            raise ValueError("output_weight_bound should be non-negative")
            
        l = proliferating_layer
        for t in range(proliferating_n):
            self.ly[l].w.add_row(mcmc_normal(self.ly[l].w.v, mean=self.ly[l].w.v.mean(), std=self.ly[l].w.v.std()))
            self.ly[l].b.add_row(mcmc_normal(self.ly[l].b.v, mean=self.ly[l].b.v.mean(), std=self.ly[l].b.v.std()))
            self.ly[l+1].w.add_column(output_weight_bound*(2*np.random.rand((self.ly[l+1].nn))-1.))
            self.ly[l].nn += 1
    
    def reset_cs(self, new_cs):
        for l in range(self.ln):
            self.ly[l].reset_cs(new_cs)
    
    def inter_layer_linear_regression(self, layer_interval):
        try:
            ls = layer_interval[0] # layer start
            le = layer_interval[1] # layer end
            if (ls < le) and (ls >= 0) and (le < self.ln):
                if ls == 0:
                    ri = np.array(self.px.T) # regression input
                else:
                    ri = np.array(self.ly[ls-1].y)
                
                ri = np.append(ri, np.ones((1, ri.shape[1])), axis=0) # append 1. for each datum as bias
                ro = np.array(self.ly[le].x)
            else:
                raise ValueError("layer_interval should be list-like, two int (a, b), with 0 <= a < b < total layer")
        
        except:
            raise ValueError("layer_interval should be list-like, two int (a, b), with 0 <= a < b < total layer")
        
        rr = np.linalg.lstsq(ri.T, ro.T) # regression result (matrix, residuals, rank of ri, singuler values of ri)
        if len(rr[1]) == 0:
            raise ValueError("output data of layer" + str(ls-1) + "(= -1, for input data) should be full rank, try self.nruron_refine first")
        
        return rr[0], rr[1]/ri.shape[1]
    
    def find_linearist_layers(self, reference_data = None):
        output = (0, 0, np.inf, np.array([[]]), np.zeros((0,0)))
        if type(reference_data) == type(None):
            self.prediction(self.tx)
        else:
            self.prediction(reference_data)
        
        for l1 in range(self.ln-1):
            for l2 in range(i+1, self.ln):
                rr = self.inter_layer_linear_regression((l1,l2))
                if np.sqrt(rr[1].sum()) < output[2]:
                    output = (l1, l2, np.sqrt(rr[1].sum()), rr[0])
        
        return output
    
    def layer_filled(self, layer_interval, weights, bias):
        try:
            ls = layer_interval[0] # layer start
            le = layer_interval[1] # layer end
            if (ls < le) and (ls >= 0) and (le < self.ln):
                pass
            else:
                raise ValueError("layer_interval should be list-like, two int (a, b), with 0 <= a < b < total layer")
        except:
            raise ValueError("layer_interval should be list-like, two int (a, b), with 0 <= a < b < total layer")
        
        if weights.shape[0] != bias.shape[0]:
            raise ValueError("weights.shape[0] doesn't match bias.shape[0]")
        
        if weights.shape[0] != self.ly[le].nn:
            raise ValueError("weights.shape[0] doesn't match #neuron of layer at end of layer_interval")
        
        self.ly[le].w.assign_values(weights)
        self.ly[le].b.assign_values(bias)
        self.ly = self.ly[:ls] + self.ly[le:]
        self.ln = len(self.ly)
    
    def linear_filled(self, layer_interval):
        try:
            ls = layer_interval[0] # layer start
            le = layer_interval[1] # layer end
            if (ls < le) and (ls >= 0) and (le < self.ln):
                pass
            else:
                raise ValueError("layer_interval should be list-like, two int (a, b), with 0 <= a < b < total layer")
        except:
            raise ValueError("layer_interval should be list-like, two int (a, b), with 0 <= a < b < total layer")
            
        rr = self.inter_layer_linear_regression(layer_interval)
        self.layer_filled(layer_interval, rr[0].T[:,:-1], rr[0].T[:,-1:])
    
    def insert_layer(self, position, weights, bias, activation_function, next_layer_weights, next_layer_bias):
        if type(position) == int:
            if position in range(self.ln):
                pass
        else:
            raise ValueError("position should be int between 0 to self.ln")
        
        if type(activation_function) == type:
            raise TypeError("activation_function should be a class. eg: Use 'Sigmoid()', not 'Sigmoid'")
        
        ilo, ili = weights.shape # input and output size of inserted layer
        nlo, nli = next_layer_weights.shape # input and output size of next layer
        
        if position == 0:
            if ili != self.xs:
                raise ValueError("weights.shape error, cheak input and output size for this new layer")
        else:
            if ili != self.ly[position-1].nn:
                raise ValueError("weights.shape error, cheak input and output size for this new layer")
        
        if (ilo != bias.shape[0]) or (ilo != nli):
            raise ValueError("to define #neuron of new layer, all related weighs and bias size should be consistent")
        
        if nlo != self.ly[position].nn:
            raise ValueError("next_layer_weights.shape error, cheak #neuron of next layer")
        
        if next_layer_bias.shape[0] != self.ly[position].nn:
            raise ValueError("next_layer_bias.shape error, cheak #neuron of next layer")
        
        if (bias.shape[1] != 1) or (next_layer_bias.shape[1] != 1):
            raise ValueError("bias shape should be (#neuron, 1)")
        
        l = position
        
        self.ly.insert(l, Layer(ilo, activation_function))
        self.ly[l].w.assign_values(weights)
        self.ly[l].b.assign_values(bias)
        self.ly[l+1].w.assign_values(next_layer_weights)
        self.ly[l+1].b.assign_values(next_layer_bias)
        
        self.ln = len(self.ly)
    
    def identity_dig(self, position, activation_function):
        if type(position) == int:
            if position in range(self.ln):
                pass
            else:
                raise ValueError("position should be int between 0 to self.ln")
        else:
            raise ValueError("position should be int between 0 to self.ln")
        
        if type(activation_function) == type:
            raise TypeError("activation_function should be a class. eg: Use 'Sigmoid()', not 'Sigmoid'")
        
        l = position
        # ids : size of identity transform, input size of new layer
        if l == 0:
            ids = self.xs
        else:
            ids = self.ly[l-1].nn
        
        if type(activation_function) in [Relu, SoftPlus]:
            liw = np.concatenate((np.identity(ids), -np.identity(ids)), axis = 0)
            lib = np.zeros((2*ids, 1))
            low = np.concatenate((np.identity(ids), -np.identity(ids)), axis = 1)
            lob = np.zeros((ids, 1))
        elif type(activation_function) == LeakyRelu:
            liw = np.concatenate((np.identity(ids), -np.identity(ids)), axis = 0)
            lib = np.zeros((2*ids, 1))
            low = np.concatenate((np.identity(ids), -np.identity(ids)), axis = 1) / (1.+activation_function.alpha)
            lob = np.zeros((ids, 1))
        elif type(activation_function) == Identity:
            liw = np.identity(ids)
            lib = np.zeros((2*ids, 1))
            low = np.identity(ids)
            lob = np.zeros((ids, 1))
        elif type(activation_function) in [Sigmoid, Hypertan, Selu]:
            # li : input of new layer
            if l == 0:
                li = np.array(self.tx.T)
            else:
                li = np.array(self.ly[l-1].y)
            
            lim = li.mean(axis=1)
            lis = li.std(axis=1) + 1.
            
            liw = np.diag(1./lis)
            if type(activation_function) == Selu:
                lib = 1.-(lim/lis).reshape(-1,1) # let mean become one before transform by activation function
            else:
                lib = -(lim/lis).reshape(-1,1) # let mean become zero before transform by activation function
            
            lo = activation_function.trans(np.dot(liw, li)+lib)
            lo = np.append(lo, np.ones((1, lo.shape[1])), axis=0) # append 1. for each datum as bias
            rr = np.linalg.lstsq(lo.T, li.T) # regression result (matrix, residuals, rank of ri, singuler values of ri)
            # since the goal is construct identity, try to find linear transform form layer output to layer input
            low = rr[0].T[:,:-1]
            lob = rr[0].T[:,-1:]
        else:
            raise TypeError("activation_function type error")
        
        nlw = np.dot(self.ly[l].w.v, low)
        nlb = np.dot(self.ly[l].w.v, lob) + self.ly[l].b.v
        
        self.insert_layer(l,
                          liw,
                          lib,
                          activation_function,
                          nlw,
                          nlb
                         )
    
    def dimension(self):
        output = 0
        for l in range(self.ln):
            output += self.ly[l].dimension()
        
        return output
    
    def save_weight(self, dir_name):
        for l in range(self.ln):
            np.save(dir_name + "/w%i.npy" % l, self.ly[l].w.v)
            np.save(dir_name + "/b%i.npy" % l, self.ly[l].b.v)
    
    def load_weight(self, dir_name):
        for l in range(self.ln):
            try:
                if l == 0:
                    if np.load(dir_name + "/w%i.npy" % l).shape[1] != self.xs:
                        raise ValueError("layer %i input size error, cheak weight size." % l)
                else:
                    if np.load(dir_name + "/w%i.npy" % l).shape[1] != self.ly[l-1].nn:
                        raise ValueError("layer %i input size error, cheak weight size." % l)

                if np.load(dir_name + "/w%i.npy" % l).shape[0] != self.ly[l].nn:
                    raise ValueError("layer %i neuron size error, cheak weight size." % l)

                if np.load(dir_name + "/b%i.npy" % l).shape[0] != self.ly[l].nn:
                    raise ValueError("layer %i neuron size error, cheak bias size." % l)

                if np.load(dir_name + "/b%i.npy" % l).shape[1] != 1:
                    raise ValueError("layer %i bias size error, should be 1." % l)
            
            except:
                raise ValueError("load .npy error, cheak dir.")
            
            self.ly[l].w.assign_values(np.load(dir_name + "/w%i.npy" % l))
            self.ly[l].b.assign_values(np.load(dir_name + "/b%i.npy" % l))

"""

"done"

'done'

In [6]:
len(set([1,2,2,5]))

3